key1: VOVS4AVLLI4O60VB  
key2: FL9L3QZH98T5M475  
key3: VHR4EUBFI6V8OWBN  
key4: NLLJW4Y93AVKHNNR  
key5: V9UWM1TL37HYVQ31

<!-- Data update -->

In [ ]:
import requests
import csv
from datetime import datetime, timedelta
import os

# Define the symbol
SYMBOL = "IBM"  # Replace with your desired stock symbol

# Paths to the dataset CSV files
CSV_FILE = "dataset_daily.csv"
TAIL_CSV_FILE = "tail_daily.csv"

# Function to fetch data from Alpha Vantage API
def fetch_data(symbol, apikey):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={apikey}"
    response = requests.get(url)
    data = response.json()
    if "Error Message" in data:
        print("Error:", data["Error Message"])
        return None
    return data.get("Time Series (Daily)", {})

# Function to read the last date from the CSV file
def get_last_date(csv_file):
    try:
        with open(csv_file, mode="r") as file:
            reader = csv.DictReader(file)
            rows = list(reader)
            if not rows:
                return None
            last_row = rows[-1]
            return datetime.strptime(last_row["Date"], "%Y-%m-%d")
    except FileNotFoundError:
        print(f"File {csv_file} not found. Starting from scratch.")
        return None

# Function to initialize the CSV file with headers if it does not exist
def initialize_csv(csv_file):
    if not os.path.exists(csv_file):
        with open(csv_file, mode="w", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=["Date", "Open", "High", "Low", "Close", "Volume"])
            writer.writeheader()

# Function to append new data to a CSV file
def append_to_csv(csv_file, new_data):
    with open(csv_file, mode="a", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["Date", "Open", "High", "Low", "Close", "Volume"])
        writer.writerows(new_data)

# Main script
if __name__ == "__main__":
    # Initialize both CSV files to ensure headers are present
    initialize_csv(CSV_FILE)
    initialize_csv(TAIL_CSV_FILE)

    # Get the last date in the primary CSV file
    last_date = get_last_date(CSV_FILE)

    # If no last date, start from the current date minus 1 year (or any default range)
    if not last_date:
        last_date = datetime.now() - timedelta(days=365)

    # Fetch the latest data from the API
    time_series = fetch_data(SYMBOL, API_KEY)
    if not time_series:
        exit()

    # Filter new data (dates after the last date in the primary CSV)
    new_data = []
    for date, values in time_series.items():
        date_obj = datetime.strptime(date, "%Y-%m-%d")
        if date_obj > last_date:
            new_data.append({
                "Date": date,
                "Open": values["1. open"],
                "High": values["2. high"],
                "Low": values["3. low"],
                "Close": values["4. close"],
                "Volume": values["5. volume"]
            })

    # Sort new data by date (oldest first)
    new_data.sort(key=lambda x: x["Date"])

    # Append new data to both CSV files
    if new_data:
        append_to_csv(CSV_FILE, new_data)
        append_to_csv(TAIL_CSV_FILE, new_data)
        print(f"Added {len(new_data)} new rows to {CSV_FILE} and {TAIL_CSV_FILE}.")
    else:
        print("No new data to add.")